<center><h1>Singh_Vasundhara_HW3</h1></center>
<br>
<br>

Name: Vasundhara Singh
<br>
Github Username: vasu1406
<br>
USC ID: 7421414643
<br>
Late By 1 Day

## 1. Time Series Classification Part 1: Feature Creation/Extraction

### (a) Download Data

Package imports

In [1]:
import os
import pandas as pd
import numpy as np
from scipy.stats import bootstrap
from sklearn.linear_model import LogisticRegression as logisticRegression

Get the AReM Data Set

In [2]:
def readCSVToDataSet(folder):
    folderPath ='../Data/AReM/'+folder+'/'
    listToAdd = [1,2,3]
    if(folder == 'bending1' or folder == 'bending2'):
        listToAdd = [1,2]
    else:
        listToAdd = [1,2,3]
    
#     print(folder)
#     print(listToAdd)
    testList = [f"dataset{x}.csv" for x in listToAdd]
    allfiles = os.listdir(folderPath)
#     print(allfiles)
    trainingData = []
    testData = []
    
    for file in allfiles:
        path = os.path.join(folderPath, file)
#         print(path)
        df = pd.read_csv(path,skiprows=range(0, 4))
#         display(df)
        if file in testList:
            testData.append(df)
        else:
            trainingData.append(df)
            
    return trainingData,testData

### (b) Test and Train Data

In [3]:
trainingDataMap = dict()
testDataMap = dict()

folders = ['bending1','bending2','cycling','lying','sitting','standing','walking']
for folder in folders:
    dataSet = readCSVToDataSet(folder)
    trainingDataMap[folder] = dataSet[0]
    testDataMap[folder] = dataSet[1]
    

### (c) Feature Extraction

#### i. Research

Analysing the features of temporal data to make predictions and distinguish patterns constitutes of Time Series Classification.The commonly used time-domain features in time series classification are Mean, Median, Standard Deviation, Variance,Maximum,Minimum,Range, Interquartile Range,Skewness, Kurtosis, Slope,Correlation Structure, Entropy, Autocorrelation,Lagged correlation, Quartiles, Zero-crossing rate, Root Mean Square etc.


#### ii. Extraction

In [4]:
col = ['min1','max1','mean1','median1','std1','1st quart1','3rd quart1',
       'min2','max2','mean2','median2','std2','1st quart2','3rd quart2',
       'min3','max3','mean3','median3','std3','1st quart3','3rd quart3',
       'min4','max4','mean4','median4','std4','1st quart4','3rd quart4',
       'min5','max5','mean5','median5','std5','1st quart5','3rd quart5',
       'min6','max6','mean6','median6','std6','1st quart6','3rd quart6',
      'isBendingColumn']

def timeFeaturesColumns(data):
    timeFeaturesData = []
    for folder in data:
        for dataSet in data[folder]:
#             timeFeaturesData.append(timeFeaturesColumns(dataSet,folder))
        #     Removing the column header
            dataSetPrevious = dataSet.columns[1:]
            dataSetNumeric = dataSet.select_dtypes(include=[np.number])
            minimum = dataSetNumeric.min()
            maximum = dataSetNumeric.max()
            mean = dataSetNumeric.mean()
            median = dataSetNumeric.median()
            stdDev = dataSetNumeric.std()
            firstQuartile = dataSetNumeric.quantile(0.25)
            thirdQuartile = dataSetNumeric.quantile(0.75)
    
    
            timeFeatureValues = []
            for column in dataSetPrevious:
                timeFeatureValues.append(minimum[column])
                timeFeatureValues.append(maximum[column])
                timeFeatureValues.append(mean[column])
                timeFeatureValues.append(median[column])
                timeFeatureValues.append(stdDev[column])
                timeFeatureValues.append(firstQuartile[column])
                timeFeatureValues.append(thirdQuartile[column])
            if(folder == 'bending1' or  folder == 'bending2'):
                timeFeatureValues.append(1)
            else:
                timeFeatureValues.append(0)
            timeFeaturesData.append(timeFeatureValues)
    
    return pd.DataFrame(timeFeaturesData,columns = col)

In [5]:
trainingDataframe = timeFeaturesColumns(trainingDataMap)
testDataframe = timeFeaturesColumns(testDataMap)
finalDataFrame = pd.concat([trainingDataframe,testDataframe]).reset_index(drop=True)
finalDataFrame.iloc[:,:-1]

,min1,max1,mean1,median1,std1,1st quart1,3rd quart1,min2,max2,mean2,...,std5,1st quart5,3rd quart5,min6,max6,mean6,median6,std6,1st quart6,3rd quart6
0,35.00,47.40,43.954500,44.330,1.558835,43.00,45.0000,0.0,1.70,0.426250,...,1.999604,35.3625,36.500,0.0,1.79,0.493292,0.430,0.513506,0.000,0.940
1,33.00,47.75,42.179813,43.500,3.670666,39.15,45.0000,0.0,3.00,0.696042,...,3.849448,30.4575,36.330,0.0,2.18,0.613521,0.500,0.524317,0.000,1.000
2,33.00,45.75,41.678063,41.750,2.243490,41.33,42.7500,0.0,2.83,0.535979,...,2.411026,28.4575,31.250,0.0,1.79,0.383292,0.430,0.389164,0.000,0.500
3,37.00,48.00,43.454958,43.250,1.386098,42.50,45.0000,0.0,1.58,0.378083,...,2.488862,22.2500,24.000,0.0,5.26,0.679646,0.500,0.622534,0.430,0.870
4,36.25,48.00,43.969125,44.500,1.618364,43.31,44.6700,0.0,1.50,0.413125,...,3.318301,20.5000,23.750,0.0,2.96,0.555313,0.490,0.487826,0.000,0.830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,35.50,46.25,43.174938,43.670,1.989052,42.50,44.5000,0.0,2.12,0.506583,...,2.983976,12.7500,16.500,0.0,5.72,0.911979,0.830,0.666161,0.470,1.220
84,32.75,47.00,42.760563,44.500,3.398919,41.33,45.3725,0.0,3.34,0.486167,...,4.296574,13.0000,18.565,0.0,5.73,0.842271,0.710,0.722165,0.430,1.090
85,19.33,43.50,34.227771,35.500,4.889576,30.50,37.7500,0.0,14.50,3.995729,...,3.092094,14.7500,18.670,0.0,9.74,3.394125,3.100,1.792090,2.105,4.425
86,12.50,45.00,33.509729,34.125,4.850923,30.50,36.7500,0.0,13.05,4.450771,...,3.133564,14.6275,18.750,0.0,8.96,3.378479,3.085,1.787360,2.060,4.440


#### iii. Standard Deviation

In [6]:
standard_deviation = finalDataFrame.std()
standard_deviation[:-1]

min1          9.569975
max1          4.394362
mean1         5.335718
median1       5.440054
std1          1.772153
1st quart1    6.153590
3rd quart1    5.138925
min2          0.000000
max2          5.062729
mean2         1.574164
median2       1.412244
std2          0.884105
1st quart2    0.946386
3rd quart2    2.125266
min3          2.956462
max3          4.875137
mean3         4.008380
median3       4.036396
std3          0.946710
1st quart3    4.220658
3rd quart3    4.171628
min4          0.000000
max4          2.183625
mean4         1.166114
median4       1.145586
std4          0.458242
1st quart4    0.843620
3rd quart4    1.552504
min5          6.124001
max5          5.741238
mean5         5.675593
median5       5.813782
std5          1.024898
1st quart5    6.096465
3rd quart5    5.531720
min6          0.045838
max6          2.518921
mean6         1.154812
median6       1.086474
std6          0.517617
1st quart6    0.758584
3rd quart6    1.523599
dtype: float64

In [9]:
confidenceInterval = dict()
for col in finalDataFrame.columns[:-1]:
    cf = bootstrap((finalDataFrame[col],), np.std, confidence_level=0.90,
                         random_state=1, method='percentile').confidence_interval
    confidenceInterval[col] = cf

confidenceInterval

{'min1': ConfidenceInterval(low=8.220030583677477, high=10.709495603487852),
 'max1': ConfidenceInterval(low=3.314995620842594, high=5.2855270712546805),
 'mean1': ConfidenceInterval(low=4.68118629427501, high=5.849899034669991),
 'median1': ConfidenceInterval(low=4.76602008649444, high=5.967149662012756),
 'std1': ConfidenceInterval(low=1.5630191079436122, high=1.9370584655327387),
 '1st quart1': ConfidenceInterval(low=5.5435084266101775, high=6.603828876321537),
 '3rd quart1': ConfidenceInterval(low=4.307611950806717, high=5.807620045905693),
 'min2': ConfidenceInterval(low=0.0, high=0.0),
 'max2': ConfidenceInterval(low=4.606956668619025, high=5.371280286178949),
 'mean2': ConfidenceInterval(low=1.3898556652290928, high=1.6960694764705273),
 'median2': ConfidenceInterval(low=1.2324907822015692, high=1.535499793042144),
 'std2': ConfidenceInterval(low=0.7988282897194028, high=0.9370162792471377),
 '1st quart2': ConfidenceInterval(low=0.8263288170602067, high=1.030598676218863),
 '3rd

#### iv. Select Features

In [10]:
classifier = logisticRegression(random_state = 42, max_iter=1000).fit(finalDataFrame.iloc[:,:-1], finalDataFrame['isBendingColumn'])
coefficients = classifier.coef_
# print(coefficients)

coeff_dict = dict(zip(finalDataFrame.columns[:-1], coefficients[0]))
coeff_dict

featuresCount = 6
minSum = 0
maxSum = 0
meanSum = 0 
medianSum = 0
stdSum = 0
firstQuartileSum = 0
thirdQuartileSum = 0

for k,v in coeff_dict.items():
    if(k == 'min1' or k == 'min2' or k == 'min3' or k == 'min4' or k == 'min5' or k == 'min6'):
        minSum += abs(v)
    elif (k == 'max1' or k == 'max2' or k == 'max3' or k == 'max4' or k == 'max5' or k == 'max6'):
        maxSum += abs(v)
    elif (k == 'mean1' or k == 'mean2' or k == 'mean3' or k == 'mean4' or k == 'mean5' or k == 'mean6'):
        meanSum += abs(v)
    elif (k == 'median1' or k == 'median2' or k == 'median3' or k == 'median4' or k == 'median5' or k == 'median6'):
        medianSum += abs(v)
    elif(k == 'std1' or k == 'std2' or k == 'std3' or k == 'std4' or k == 'std5' or k == 'std6'):
        stdSum += abs(v)
    elif(k == '1st quart1' or k == '1st quart2' or k == '1st quart3' or k == '1st quart4' or k == '1st quart5' or k == '1st quart6'):
        firstQuartileSum += abs(v)
    else:
        thirdQuartileSum += abs(v)
        

print('min_avg : ',(minSum/featuresCount))
print('max_avg : ',(maxSum/featuresCount))
print('mean_avg : ',(meanSum/featuresCount))
print('median_avg : ',(medianSum/featuresCount))
print('standardDeviation_avg : ',(stdSum/featuresCount))
print('firstQuartile_avg : ',(firstQuartileSum/featuresCount))
print('thirdQuartile_avg : ',(thirdQuartileSum/featuresCount))

min_avg :  0.10137056255594719
max_avg :  0.19634376032101034
mean_avg :  0.10833168743376688
median_avg :  0.08695851306737325
standardDeviation_avg :  0.028390269193569295
firstQuartile_avg :  0.07585152017740815
thirdQuartile_avg :  0.1435763978737987


I fitted a Logisitic Regression model to all the time-domain features and the target variable/class = Bending(isBendingColumn = 1)  or not Bending (isBendingColumn = 0). The averaged printed above are the averages of all the coefficients of the time-domain features. And as we can see the time-domain features **Max, 3rd Quartile and Mean** have the highest average. Hence these seem to be the most important time domain features.

## 2. ISLR 3.7.4

### (a) Linear Train

The **training RSS for cubic regression would be lower than the training RSS for linear regression**. Even if the true relation between X and Y is linear, the cubic regression would fit closer to some erroneous data points because of **overfitting** and hence give lower RSS than the linear regression.

### (b) Linear Test

The **test RSS for linear regression would be lower than the cubic regression**. Since the true relation between X and Y is linear, linear regression would fit better to the underlying model. On the other hand the higher RSS for cubic regression would be attributed to **Overfitting** and it's lowered ability to generalise.


### (c) Not Linear Train

The **train RSS for cubic regression will be lower than the RSS of linear regression**. As explained in part a) the cubic regression fit closer to data points due to overfitting and hence would give lower RSS if the underlying relationship is not linear

### (d) Not Linear Testing

**Not enough information to conclude which regression would be better**. If the true/underlying relation is cubic or of even higher degree the test RSS for cubic regression would be lower. Whereas if the underlying relationship is close to linear relationship the test RSS for linear regression would be lower.